In [ ]:
!pip install pymongo

In [ ]:
%load_ext google.colab.data_table
import pymongo
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime, timedelta
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.model_selection import LeaveOneOut, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
import sklearn
 
def corregir_dtypes_dataframe(df):
    df.cantidad = pd.to_numeric(df.cantidad)
    return df

 
def crear_dataframe_desde_mongodb():
    cadena_conexion = ('mongodb://{nom_usuario}:{password}@{host}:{port}')
    cadena_conexion = cadena_conexion.format(
        nom_usuario = 'PY01_c02',
        password = 'P4rd83XkXrTz',
        host = '5.189.129.12',
        port = 27017
    )
    client = pymongo.MongoClient(cadena_conexion)
    db = client.PY01
    header = [*db.PY01.find_one().keys()]
    lista_dataset = list()
    for doc in db.PY01.find():
        lista_dataset.append([*doc.values()])
 
    df = pd.DataFrame(data = lista_dataset, columns = header)
    return df
 
def crear_codigos_prod(serie):
    prods = serie.unique()
    cantidad_prods = len(prods)
    prods = sorted(prods)
    return dict(zip(prods, range(cantidad_prods)))
 
def limpiar_dataset(df):
    dias = {'LUNES': 1, 'MARTES': 2, 'MIERCOLES': 3, 'JUEVES': 4, 'VIERNES': 5, 'SABADO': 6, 'SUNDAY': 7}
    for dia, cod in dias.items():
        df.loc[df.dia == dia, 'dia'] = cod
    df.dia = pd.to_numeric(df.dia)
    
    for prod, cod in crear_codigos_prod(df['prod']).items():
        df.loc[df['prod'] == prod, 'prod'] = cod
    return df
 
def generar_variables(df):
    nuevos_registros = list()
    for row in df.itertuples():
        cantidad = row.cantidad
        if cantidad > 1:
            cantidad -= 1
            for t in range(cantidad):
                nuevos_registros.append(row)
 
    df = df.append(nuevos_registros, ignore_index = True).drop(labels = ['Index','cantidad'], axis = 1)
 
    df = pd.concat((df, pd.get_dummies(df.dia, prefix = 'dia')), axis = 1)
    df = df.assign(dia_sin = lambda x: np.sin(2 * np.pi * x.dia / 7))
    df = df.assign(dia_cos = lambda x: np.cos(2 * np.pi * x.dia / 7))
 
    return df[['cliente','dia','dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','dia_sin','dia_cos','prod']] 
 
def preparar_dataset(df):
    df = df.drop(labels = ['_id','fecNacimiento','sexo','anho','mes','monto','descuento'], axis = 1)
    df = corregir_dtypes_dataframe(df)
    df = limpiar_dataset(df)
    df = generar_variables(df)
    return df
 
def generar_lista_variantes():
    variantes = {1: ['dia','prod'],
                 2: ['dia_1','dia_2','dia_3','dia_4','dia_5','dia_6','dia_7','prod'],
                 3: ['dia_sin','dia_cos','prod']}
    return variantes
 
def aplicar_grid_search(X, y, modelo, params):
    if len(X) > 100:
        cv = StratifiedKFold(10)
    else:
        cv = LeaveOneOut()
    gs = GridSearchCV(estimator = modelo,
                      param_grid = params,
                      cv = cv,
                      scoring = ['accuracy','f1_weighted'],
                      refit = 'f1_weighted',
                      return_train_score = True,
                      verbose = 2,
                      n_jobs = -1)
    with tf.device('/device:GPU:0'):
        gs.fit(X, y)
    return gs
 
def entrenar_LR(metodo_multiclass, X, y):
    modelo = LogisticRegression()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__C = np.logspace(-6,2,9), estimator__solver = ('newton-cg','sag','saga','lbfgs'))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_SVM(metodo_multiclass, X, y):
    modelo = SVC()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__C = np.logspace(-6,2,9), estimator__gamma = np.logspace(-6,2,9))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_DT(metodo_multiclass, X, y):
    modelo = DecisionTreeClassifier()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__max_depth = range(2,11,2), estimator__min_samples_split = range(2,11,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_NB(metodo_multiclass, X, y):
    modelo = GaussianNB()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__var_smoothing = np.logspace(-11,-7,5))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar_KNN(metodo_multiclass, X, y):
    modelo = KNeighborsClassifier()
    if metodo_multiclass is not None: modelo = metodo_multiclass(modelo)
    params = dict(estimator__n_neighbors = range(2, 31, 3), estimator__weights = ('uniform','distance'), estimator__p = (1,2))
    clasificador = aplicar_grid_search(X, y, modelo, params)
    return clasificador

def entrenar(df, variantes):
    lista_clientes = sorted(df.cliente.unique(), key = lambda x: int(x[3:]))
    lista_clientes.remove('SID1')
    modelo_cliente = list()
    for cliente in lista_clientes:
        print(cliente)
        clasificadores = list()
        for cod_variante, variante in variantes.items():
            base = df[df.cliente == cliente][variante].copy()
            X = base.drop(labels = 'prod', axis = 1)
            y = base['prod'].astype('int')
            if len(y.unique()) > 1:
                for metodo_multiclass in (OneVsOneClassifier, OneVsRestClassifier):
                    for entrenar_algoritmo in (entrenar_NB, entrenar_DT, entrenar_LR, entrenar_SVM):
                        clasificadores.append((cliente, entrenar_algoritmo(metodo_multiclass, X, y)))
            else:
                clasificadores.append((cliente, y.iloc[0]))
        modelo_cliente.append(clasificadores)
    return modelo_cliente
 
def main():
    df = crear_dataframe_desde_mongodb()
    df = preparar_dataset(df)
    variantes = generar_lista_variantes()
    c = entrenar(df, variantes)
    return c
 
np.random.seed(10)
ans = main()

SID2
Fitting 33 folds for each of 5 candidates, totalling 165 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 165 out of 165 | elapsed:   29.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 33 folds for each of 25 candidates, totalling 825 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:   44.0s
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 825 out of 825 | elapsed:  1.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 33 folds for each of 36 candidates, totalling 1188 fits


...

Fitting 10 folds for each of 25 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  2.4min finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.2min


KeyboardInterrupt: ignored

In [ ]:
import pickle
pickle_out = open('trabajo.pkl', 'wb')
pickle.dump(ans, pickle_out)
pickle_out.close()

In [ ]:
!pip install scikit-learn==0.23.2 --upgrade

     |████████████████████████████████| 6.8MB 5.1MB/s 
     |████████████████████████████████| 307kB 34.7MB/s 
     |████████████████████████████████| 15.3MB 231kB/s 
     |████████████████████████████████| 27.4MB 96kB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: joblib 1.0.1
    Uninstalling joblib-1.0.1:
      Successfully uninstalled joblib-1.0.1
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.

In [ ]:
import pickle
pickle_in = open('trabajo.pkl', 'rb')
ans = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
from sklearn.model_selection import GridSearchCV
clientes_cod = {1:'SID2',2:'SID3',3:'SID5',4:'SID6',5:'SID7',6:'SID8',7:'SID9',8:'SID10',9:'SID12',10:'SID13',11:'SID14',12:'SID15',13:'SID17',14:'SID18',15:'SID20',16:'SID21',17:'SID22',18:'SID23',19:'SID24',20:'SID25',21:'SID26',22:'SID27',23:'SID28',24:'SID29',25:'SID30',26:'SID32',27:'SID33',28:'SID34',29:'SID35',30:'SID36',31:'SID38',32:'SID39',33:'SID40',34:'SID41',35:'SID42',36:'SID43',37:'SID44',38:'SID45',39:'SID46',40:'SID47',41:'SID48',42:'SID49',43:'SID50',44:'SID52',45:'SID53',46:'SID54',47:'SID55',48:'SID56',49:'SID57',50:'SID59',51:'SID61',52:'SID65',53:'SID66',54:'SID67',55:'SID68',56:'SID70',57:'SID71',58:'SID72',59:'SID73',60:'SID74',61:'SID75',62:'SID76',63:'SID77',64:'SID78',65:'SID79',66:'SID80',67:'SID81',68:'SID82',69:'SID83',70:'SID84',71:'SID85',72:'SID86',73:'SID87',74:'SID88',75:'SID89',76:'SID90',77:'SID91',78:'SID92',79:'SID93'}
n_features_variantes = {1: 1, 7: 2, 2: 3}
clasificadores = dict()
for cod_cliente, cliente in enumerate(ans):
    print(clientes_cod[cod_cliente + 1])
    if type(cliente[0]) == GridSearchCV:
        mejor_gs = sorted(cliente, key = lambda gs: (gs.best_score_, gs.cv_results_['mean_test_accuracy'][gs.best_index_], gs.cv_results_['mean_train_f1_weighted'][gs.best_index_], gs.cv_results_['mean_train_accuracy'][gs.best_index_]), reverse = True)[0]
        mejor_clasificador = mejor_gs.best_estimator_
        variante = n_features_variantes[mejor_gs.n_features_in_]
    else:
        mejor_clasificador = cliente[0]
        variante = None
    clasificadores[clientes_cod[cod_cliente + 1]] = (variante, mejor_clasificador)
    

SID2
SID3
SID5
SID6
SID7
SID8
SID9
SID10
SID12
SID13
SID14
SID15
SID17
SID18
SID20
SID21
SID22
SID23
SID24
SID25
SID26
SID27
SID28
SID29
SID30
SID32
SID33
SID34
SID35
SID36
SID38
SID39
SID40
SID41
SID42
SID43
SID44
SID45
SID46
SID47
SID48
SID49
SID50
SID52
SID53
SID54
SID55
SID56
SID57
SID59
SID61
SID65
SID66
SID67
SID68
SID70
SID71
SID72
SID73
SID74
SID75
SID76
SID77
SID78
SID79
SID80
SID81
SID82
SID83
SID84
SID85
SID86
SID87
SID88
SID89
SID90
SID91
SID92
SID93


In [ ]:
clasificadores

{'SID10': (2,
  OneVsRestClassifier(estimator=LogisticRegression(C=1e-06, solver='sag'))),
 'SID12': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID13': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID14': (2,
  OneVsOneClassifier(estimator=LogisticRegression(C=1e-06, solver='sag'))),
 'SID15': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID17': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID18': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID2': (3,
  OneVsOneClassifier(estimator=LogisticRegression(C=100.0, solver='newton-cg'))),
 'SID20': (None, 205),
 'SID21': (1,
  OneVsOneClassifier(estimator=DecisionTreeClassifier(max_depth=2))),
 'SID22': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID23': (1, OneVsOneClassifier(estimator=GaussianNB(var_smoothing=1e-11))),
 'SID24': (1, OneVsOneClassifier(estimator=GaussianNB

In [ ]:
pickle_out = open('clasificadores.pkl', 'wb')
pickle.dump(clasificadores, pickle_out)
pickle_out.close()